You now know the following

1. Generate open-loop control from a given route

2. Simulate vehicular robot motion using bicycle/ unicycle model

Imagine you want to make an utility for your co-workers to try and understand vehicle models. 
Dashboards are common way to do this.

There are several options out there : Streamlit, Voila, Observable etc

Follow this
<a href="https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e">Medium post</a> on Jupyter Dash and see how to package what you learnt today in an interactive manner

Here is a   <a href="https://stackoverflow.com/questions/53622518/launch-a-dash-app-in-a-google-colab-notebook">stackoverflow question </a> on how to run dash applications on Collab

What can you assume?
+ Fix $v,\omega$ or $v,\delta$ depending on the model (users can still pick the actual value)
+ fixed wheelbase for bicycle model

Users can choose 
+ unicycle and bicycle models
+ A pre-configured route ("S", "inverted-S", "figure-of-eight" etc)
+ 1 of 3 values for $v, \omega$ (or $\delta$) 

In [1]:
!pip install jupyter-dash

     |████████████████████████████████| 81kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 4.6MB/s 
     |████████████████████████████████| 3.5MB 7.9MB/s 
     |████████████████████████████████| 92kB 9.9MB/s 
     |████████████████████████████████| 1.8MB 22.9MB/s 
     |████████████████████████████████| 358kB 35.3MB/s 
  Created wheel for dash: filename=dash-1.20.0-cp37-none-any.whl size=85845 sha256=2ecb66d28a1d121d78e0d21b9094c74fb6e54b25b60f7bd86df807da79028294
  Stored in directory: /root/.cache/pip/wheels/e6/33/41/ce361d0d1da277d2350f815b10f2ab1e331592d93ef2b07e29
  Created wheel for dash-renderer: filename=dash_renderer-1.9.1-cp37-none-any.whl size=1014873 sha256=0f4f661e6aef7c439b0a262187f7fa5fdf7301120d79948ae647f7c29e016d3b
  Stored in directory: /root/.cache/pip/wheels/03/a9/c5/dd5815c601b0ede164c223ffd7bafebde716ca57de06ef8aec
  Created wheel for dash-core-components: filename=dash_core_components-1.16.0-cp37-none-any.whl size=3540992 sha256=589d954813924a3249428c78

In [3]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np

# Load Data
vs = ['1','2','3']
ws = ['15','30','45']
routes = ["S", "Inverted-S", "Figure of 8"]
cycles = ["Unicycle", "Bicycle"]

def unicycle_model(curr_pose, v, w, dt=1.0):
    x, y, theta = curr_pose
    x += v*np.cos(theta)*dt
    y += v*np.sin(theta)*dt
    theta += w*dt
    theta = np.arctan2(np.sin(theta), np.cos(theta))
    return x, y, theta
  
def bicycle_model(curr_pose, v, delta, dt=1.0):
    L = 0.9
    x, y, theta = curr_pose 
    x += v*np.cos(theta)*dt 
    y += v*np.sin(theta)*dt
    theta += (v/L)*np.tan(delta)*dt
    theta = np.arctan2(np.sin(theta), np.cos(theta))
    return x, y, theta

def get_open_loop_commands(route, vc_fast=1, wc=np.pi/12, dt=1.0):
    all_w = []
    omegas = {'straight': 0, 'left': wc, 'right': -wc}
    for manoeuvre, command in route:
      u = np.ceil(command/vc_fast).astype('int')
      v = np.ceil(np.deg2rad(command)/wc).astype('int')
      t_cmd = u if manoeuvre == 'straight' else v
      all_w += [omegas[manoeuvre]]*t_cmd
    all_v = vc_fast * np.ones_like(all_w)
    return all_v, all_w

# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Kinemetic Model"),
    dcc.Graph(id='graph'),
    html.Label([
        "v",
        dcc.Dropdown(
            id='v', clearable=False,
            value='1', options=[
                {'label': c, 'value': c}
                for c in vs
            ])
    ]),
    html.Label([
        "o(or d)",
        dcc.Dropdown(
            id='o', clearable=False,
            value='15', options=[
                {'label': c, 'value': c}
                for c in ws
            ])
    ]),
    html.Label([
        "Route",
        dcc.Dropdown(
            id='route', clearable=False,
            value='S', options=[
                {'label': c, 'value': c}
                for c in routes
            ])
    ]),
    html.Label([
        "Unicycle/Bicycle",
        dcc.Dropdown(
            id='ub', clearable=False,
            value='Unicycle', options=[
                {'label': c, 'value': c}
                for c in cycles
            ])
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("v", "value"), Input("o", "value"), Input("route", "value"), Input("ub", "value")]
)

def update_figure(velocity, omega, shape, model):
    robot_trajectory = []
    route = None
    if(shape == routes[0]):
        route =  [("right", 180),("left", 180)]
    elif(shape == routes[1]):
        route = [("left", 180),("right", 180)]
    else:
        route = [("right", 180),("left", 180),("left", 180),("right", 180)]
    angle = np.pi
    if(omega == ws[0]):
        angle = np.pi/12
    elif(omega == ws[1]):
        angle = np.pi/6
    else:
        angle = np.pi/4
    all_v, all_w = get_open_loop_commands(route, int(velocity), angle)
    pose = (0, 0, np.pi/2)
    robot_trajectory.append(pose)
    for v, w in zip(all_v, all_w):
        if model == cycles[0]:
          pose = unicycle_model(pose, v, w)
        else:
          pose = bicycle_model(pose,v,w)
        robot_trajectory.append(pose)
    robot_trajectory = np.array(robot_trajectory)
    dt = pd.DataFrame({'y': robot_trajectory[:,0],'x': robot_trajectory[:,1]})
    return px.line(dt, y="y", x="x", title='Simulate vehicular robot motion')
# Run app and display result inline in the notebook
app.run_server(mode='inline')

<IPython.core.display.Javascript object>